In [14]:
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
from langchain_google_vertexai import ChatVertexAI
from typing import TypedDict
from langgraph.checkpoint.memory import InMemorySaver

In [15]:
load_dotenv()

llm=ChatVertexAI(model_name="gemini-2.5-pro")

In [16]:
class JokeState(TypedDict):
    topic:str
    joke:str
    explaination:str

In [17]:
def generate_joke(state:JokeState):
    prompt=f"generate a joke on the topic {state['topic']}"
    response=llm.invoke(prompt).content
    return {'joke':response}

In [18]:
def generate_explaination(state:JokeState):
    prompt=f"Write an explaination for the joke - {state["joke"]}"
    response=llm.invoke(prompt).content
    return {"explaination":response}

In [20]:
graph=StateGraph(JokeState)

graph.add_node('generate_joke',generate_joke)
graph.add_node('generate_explaination',generate_explaination)

graph.add_edge(START,'generate_joke')
graph.add_edge('generate_joke','generate_explaination')
graph.add_edge('generate_explaination',END)

checkpointer=InMemorySaver()
workflow=graph.compile(checkpointer=checkpointer)

In [ ]:
config1={"configurable":{'thread_id':'1'}}
workflow.invoke({'topic':"Pizza"},config=config1)